In [34]:
import Pkg; 
Pkg.activate("Conf_Fluids")
Pkg.add("Symbolics")
using Symbolics
using StaticArrays

 Activating environment at `~/Julia/Fluidos_Conformes/Conf_Fluids/Project.toml`
  Resolving package versions...
No Changes to `~/Julia/Fluidos_Conformes/Conf_Fluids/Project.toml`
No Changes to `~/Julia/Fluidos_Conformes/Conf_Fluids/Manifest.toml`


In [109]:
function F(flu,con,χ)
    f = Dict(:μ => 1, :v => 2, :x1 => 3, :x2 => 4,:x3 => 5) #para acordarse de que campo asignamos a cada componente del vector
    c = Dict(:e => 1, :s => 2, :c1 => 3, :c2 => 4,:c3 => 5)
#@variables flu[1:N], con[1:N], χ[1:3] #variables para manipulación algebraica
#@variables χ₀, χ₁, χ₂
#@variables 

    μ = flu[f[:μ]]  # esto es -μ
    T = (μ)^(-1//2) # use μ positive, so I changed μ -> -μ
    v = flu[f[:v]]
    χ₀ = χ[1]
    χ₁ = χ[2]
    χ₂ = χ[3]
    x1 = flu[f[:x1]]
    x2 = flu[f[:x2]]
    x3 = flu[f[:x3]]
    γ = (1 - v^2)^(-1//2)
    τ = -2χ₁ * x3 * T / (γ*μ^3) - 24χ₂*(1//2*(1-v^2)x3^2 + 14//3 * x2^2 + 7/5*(1-v^2)x1*x3)/μ^5
    ρ = -6χ₀ / μ^2 - 6χ₁*x1/(γ * μ^4 * T) - 42χ₂*(6//5 *x1^2 + 10γ^2*x2^2 + 3//2*(v^2-1)^2*x3^2)/(μ^5 * γ^2)
    Q = -10χ₀ * x2 * T / μ^3 - 168χ₂ * x2 * (x1 - (v^2 - 1)x3)/(γ * μ^5)

    A = (-12 * χ₂/μ^4)*[3(2γ^2 - 1)  3v*(6γ^2 - 1)    3v^2 ;
                v*(6γ^2 - 1)  (6γ^2*(1 + 2v^2) - 1)  v*(v^2 + 2) ;
                (6γ^2 * v^2 + 1)  v*(6γ^2*(2 + v^2) - 1)  (2v^2 + 1)]
#Φ₂ = -μ/12/χ₂
#Φ₂ = -u[f[:μ]]/12/χ₂
#γ = 1/sqrt(1 - u[f[:v]]^2)
    e = 4//3*ρ*(γ^2 - 1//4) + 2v*γ*Q + v^2*τ
    s = 4//3*ρ*γ^2*v + (v^2+1//1)γ*Q + v*τ;

    Y1 = -3χ₁*γ/T/μ^3 *(2γ^2 - 1//1)
    Y2 = -3χ₁*γ/T/μ^3 *v*(6γ^2 - 1//1)
    Y3 = -3χ₁*γ/T/μ^3 *(6γ^2*v^2 + 1//1)
    
    return  [- con[c[:e]] + 4//3* ρ*(γ^2 - 1//4) + 2//1* Q*γ*v + τ*v^2;
             - con[c[:s]] + 4//3* ρ*γ^2*v + Q*γ*(v^2 + 1//1) + τ*v;
             - con[c[:c1]] + A[1,1]*x1 + A[1,2]*x2 + A[1,3]*x3 + Y1;
             - con[c[:c2]] + A[2,1]*x1 + A[2,2]*x2 + A[2,3]*x3 + Y2;
             - con[c[:c3]] + A[3,1]*x1 + A[3,2]*x2 + A[3,3]*x3 + Y3]
end

F (generic function with 1 method)

In [110]:
@time F([1;0.2;1;1;1],[1;2;2;2;2],[1;1;1])

  0.000013 seconds (29 allocations: 2.375 KiB)


5-element Array{Float64,1}:
 -708.882891258125
 -535.5019804060945
  -83.5570173600189
  -91.71095528740291
  -61.987327723098716

In [111]:
N=5

5

In [112]:
@variables f[1:N], c[1:N], p[1:3]

(Num[f₁, f₂, f₃, f₄, f₅], Num[c₁, c₂, c₃, c₄, c₅], Num[p₁, p₂, p₃])

In [113]:
F(f,c,p);

In [114]:
JS = Symbolics.jacobian(F(f,c,p),f);

In [115]:
J_exp = Symbolics.build_function(JS, f, c, p);

In [116]:
Jac = eval(J_exp[1]);

In [117]:
Jac(rand(5),rand(5),rand(3))

5×5 Array{Float64,2}:
 2.63082e15  -6.43585e12  -1.79758e12  -1.02667e13  -1.00254e12
 2.38246e15  -6.4606e12   -1.7393e12   -8.94213e12  -1.02328e12
 6.80095e12  -4.97913e10  -5.54305e9   -1.37851e10  -9.5029e8
 6.47008e12  -4.85838e10  -4.59502e9   -1.36694e10  -1.17552e9
 5.81137e12  -4.71909e10  -4.06317e9   -1.18485e10  -1.37347e9

In [118]:
function NR_step!(F, Jac, u0, y, p)
    u = u0 - Jac(u0,y, p) \ F(u0,y,p)
end

NR_step! (generic function with 1 method)

In [119]:
p = [1,1,1]
u0 = @SVector[1.0, 0.30, 2.0, 4.0, 5.0]
y = F(u0,[0,0,0,0,0],p)

F(u0.+0.005.*rand(5),y ,p)

#Jac(u0,y .+0.005.*rand(5),p);
F(u0,y .+0.005.*rand(5),p)

5-element Array{Float64,1}:
 -0.0033620762733903575
 -0.0008881503725888251
 -0.0027372121371933567
 -0.0036853292062835052
 -0.004343251011975191

In [120]:
ui = @SVector[1.0, 0.30, 3.0, 4.0, 3.0]
NR_step!(F, Jac, u0, y, p)

5-element SArray{Tuple{5},Float64,1,5} with indices SOneTo(5):
 1.0000000000000002
 0.30000000000000027
 1.999999999999996
 4.0000000000000036
 5.000000000000003

In [121]:
#u0 = @SVector[1.0, 0.30, 3.0, 4.0, 3.0]
u0 = [1.0; 0.30; 3.0; 4.0; 3.0]
for i ∈ 1:25
    u0 = NR_step!(F, Jac, u0, y, p)
end
println(u0)
println(F(u0,y,p))

[1.0000000000000007, 0.3000000000000006, 1.9999999999999916, 4.000000000000007, 5.0000000000000115]
[-7.645439836778678e-12, -4.661160346586257e-12, -1.1679546219056647e-13, -1.2168044349891716e-13, -1.4477308241112041e-13]


In [122]:
M = 10^3
u0 = [(i+j)/(N+M) for i ∈ 1:N, j ∈ 1:M]
#u0 = rand(N,M)
yy = zeros(N,M)
for j ∈ 1:M
    yy[:,j] = F(u0[:,j],[0,0,0,0,0],p)
#    yy[:,j] = convert(Array{Float64,1},F(u0[:,j],yy[:,j],p))
end

In [123]:
j = 200
F(u0[:,j],yy[:,j], p)'*F(u0[:,j],yy[:,j], p)

1.2378628619885348e-23

In [124]:
u0 = (u0 .- 0.0001 .*rand(N,M))

5×1000 Array{Float64,2}:
 0.00190852  0.00294407  0.00395205  …  0.994007  0.995024  0.995957
 0.00289294  0.00393125  0.00496267     0.995022  0.996006  0.997009
 0.00389643  0.00493491  0.00592383     0.995949  0.99696   0.997979
 0.00493362  0.00588463  0.0068849      0.997001  0.99791   0.998993
 0.00588939  0.00695836  0.00789809     0.997921  0.998988  0.999942

In [125]:
tol = 10^(-8)
iter_max = 5

@time begin
for j ∈ 1:M
    iter = 1
    while F(u0[:,j],yy[:,j], p)'*F(u0[:,j],yy[:,j], p) > tol && iter < iter_max
        u0[:,j] = NR_step!(F, Jac, u0[:,j], yy[:,j], p)
        iter = iter + 1
        if iter == iter_max 
            println("iter_max reached j = $j")
        end
    end
    #println(iter)
end
end

iter_max reached j = 1
iter_max reached j = 2
iter_max reached j = 4
iter_max reached j = 6
  0.208967 seconds (505.55 k allocations: 38.495 MiB, 9.87% gc time)
